In [199]:
import psycopg2
import pandas as pd

In [200]:
#%run functions.ipynb

In [201]:
def do_query(query, print_out = False, commit = False):
    try:
        con = psycopg2.connect(
                    host = "127.0.0.11",
                    database="ZSBD_p1",
                    user = "inzdanych123",
                    password = "inzdanych123",
                    port = "5432")

        cur = con.cursor()

        cur.execute(query)
        
        for notice in con.notices:
            print(notice)

        if commit:
            con.commit()

        if print_out:
            column_names = [desc[0] for desc in cur.description]
            row_list = []
            row = cur.fetchone()
            while row is not None:
                row_list.append(row)
                #print(row)
                row = cur.fetchone()
            df = pd.DataFrame(row_list, columns=column_names)
            print(df)
        
        
        
        cur.close()
        con.close()
        print('\n')
    except (Exception, psycopg2.DatabaseError) as error:
            print(error)

    finally:
        if con is not None:
            con.close()

# FUNCTIONS

In [202]:
def function_def(func, commit = False):
    try:
        con = psycopg2.connect(
                    host = "127.0.0.11",
                    database="ZSBD_p1",
                    user = "inzdanych123",
                    password = "inzdanych123",
                    port = "5432")

        cur = con.cursor()

        cur.execute(func)
        if commit:
            con.commit()

        cur.close()
        con.close()

    except (Exception, psycopg2.DatabaseError) as error:
            print(error)

    finally:
        if con is not None:
            con.close()

In [203]:
def function_tester(function, *arguments_of_it):
    try:
        con = psycopg2.connect(
                    host = "127.0.0.11",
                    database="ZSBD_p1",
                    user = "inzdanych123",
                    password = "inzdanych123",
                    port = "5432")

        cur = con.cursor()
        
        if len(arguments_of_it) == 1:
            cur.execute(function + "('" +str(arguments_of_it[0]) + "')")
        else:
            cur.execute(function + str(arguments_of_it))

        column_names = [desc[0] for desc in cur.description]
        row_list = []
        row = cur.fetchone()
        while row is not None:
            row_list.append(row)
            #print(row)
            row = cur.fetchone()
        df = pd.DataFrame(row_list, columns=column_names)
        print(df)
        
        for notice in con.notices:
            print(notice)

        cur.close()
        con.close()

    except (Exception, psycopg2.DatabaseError) as error:
            print(error)

    finally:
        if con is not None:
            con.close()

In [204]:
function_def("""
    CREATE OR REPLACE FUNCTION ai_med.get_sum(
        a NUMERIC, 
        b NUMERIC) 
    RETURNS NUMERIC AS $$
    BEGIN
        RETURN a + b;
    END; $$

    LANGUAGE plpgsql;
    """,
    commit = True)

In [205]:
function_tester("select ai_med.get_sum", 10, 20)

  get_sum
0      30


In [206]:
function_def("""
CREATE OR REPLACE FUNCTION ai_med.test1 ()
RETURNS integer AS $total$
declare
	total integer;
BEGIN
   select count(*) into total from ai_med.pacjent;
   RETURN total;
END;
$total$ LANGUAGE plpgsql;
""",
commit = True)

In [207]:
function_tester("select ai_med.test1")

   test1
0  50091


In [208]:
do_query("""
SELECT * FROM ai_med.pacjent WHERE id = 1;
""",
print_out=True)

   id  imie nazwisko data_urodzenia                       created_at
0   1  Robo     Bobo     1934-08-31 2022-11-20 15:03:59.028075+01:00




In [321]:
do_query("""
drop function if exists ai_med.pacjenci_z_id(character, character, character);;
""", 
commit=True)

function_def("""
create or replace function ai_med.pacjenci_z_id(imieee char, nazwiskooo char, lek char)

returns table(
   data_rozpoczecie_brania_leku date,
   data_zakonczenia_brania_leku date,
   dni_brania_leku int
)

language plpgsql
as
$$

declare
   ilosc_pacjentow integer;

begin

   IF lek = 'ventolin'
      THEN
      return query
      SELECT "Aktualnie przyjmowane leki".data_rozpoczecia_ventolin__asthma
      ,"Aktualnie przyjmowane leki".data_zakonczenia_ventolin__asthma 
      ,coalesce("Aktualnie przyjmowane leki".data_zakonczenia_ventolin__asthma, cast(now() as date)) -"Aktualnie przyjmowane leki".data_rozpoczecia_ventolin__asthma as "dni_brania_ventolin"
      FROM ai_med.pacjent INNER JOIN ai_med."Aktualnie przyjmowane leki" 
      ON "Aktualnie przyjmowane leki".id = pacjent.id 
      WHERE pacjent.imie = imieee or pacjent.nazwisko = nazwiskooo;
   
   elsif lek = 'atorvastatin' 
      THEN
      return query
      SELECT "Aktualnie przyjmowane leki".data_rozpoczecia_atorvastatin_calcium__cholesterol
      ,"Aktualnie przyjmowane leki".data_zakonczenia_atorvastatin_calcium__cholesterol 
      ,coalesce("Aktualnie przyjmowane leki".data_zakonczenia_atorvastatin_calcium__cholesterol, cast(now() as date)) -"Aktualnie przyjmowane leki".data_rozpoczecia_atorvastatin_calcium__cholesterol as "dni_brania_ventolin"
      FROM ai_med.pacjent INNER JOIN ai_med."Aktualnie przyjmowane leki" 
      ON "Aktualnie przyjmowane leki".id = pacjent.id 
      WHERE pacjent.imie = imieee or pacjent.nazwisko = nazwiskooo;
   
   elsif lek = 'hydrochlorothiazide' 
      THEN
      return query
      SELECT "Aktualnie przyjmowane leki".data_rozpoczecia_hydrochlorothiazide__wysokie_cisnienie_krwi
      ,"Aktualnie przyjmowane leki".data_zakonczenia_hydrochlorothiazide__wysokie_cisnienie_krwi 
      ,coalesce("Aktualnie przyjmowane leki".data_zakonczenia_hydrochlorothiazide__wysokie_cisnienie_krwi, cast(now() as date)) -"Aktualnie przyjmowane leki".data_rozpoczecia_hydrochlorothiazide__wysokie_cisnienie_krwi as "dni_brania_ventolin"
      FROM ai_med.pacjent INNER JOIN ai_med."Aktualnie przyjmowane leki" 
      ON "Aktualnie przyjmowane leki".id = pacjent.id 
      WHERE pacjent.imie = imieee or pacjent.nazwisko = nazwiskooo;
   
   ELSE 
      RAISE EXCEPTION 'Nie ma takiego leku';
   END IF;

end;
$$;
    """,
commit = True)

In [322]:
function_tester("select * FROM ai_med.pacjenci_z_id", "Rachelle", "Wheler", "ventolin")

  data_rozpoczecie_brania_leku data_zakonczenia_brania_leku  dni_brania_leku
0                   2018-05-07                   2020-08-04              820
1                   2018-07-30                   2021-04-24              999
2                   2019-04-02                   2021-02-08              678
3                   2018-10-13                   2020-12-22              801
4                   2019-03-23                   2021-08-16              877
5                   2018-05-25                   2021-05-04             1075


In [323]:
function_tester("select * FROM ai_med.pacjenci_z_id", "Rachelle", "Wheler", "atorvastatin")

  data_rozpoczecie_brania_leku data_zakonczenia_brania_leku  dni_brania_leku
0                   2018-02-16                   2020-09-14              941
1                   2019-04-02                   2021-05-02              761
2                   2019-02-14                   2020-02-09              360
3                   2018-06-23                   2020-04-15              662
4                   2018-07-14                   2021-12-29             1264
5                   2019-01-23                   2021-01-24              732


In [324]:
function_tester("select * FROM ai_med.pacjenci_z_id", "Rachelle", "Wheler", "hydrochlorothiazide")

  data_rozpoczecie_brania_leku data_zakonczenia_brania_leku  dni_brania_leku
0                   2018-12-30                   2021-02-27              790
1                   2018-04-16                   2020-06-14              790
2                   2018-08-25                   2021-10-19             1151
3                   2018-08-13                   2021-06-01             1023
4                   2018-09-01                   2020-09-30              760
5                   2018-09-18                   2020-10-29              772


In [326]:
function_tester("select * FROM ai_med.pacjenci_z_id", "Rachelle", "Wheler", "jakislek")

Nie ma takiego leku
CONTEXT:  PL/pgSQL function ai_med.pacjenci_z_id(character,character,character) line 39 at RAISE



In [216]:
function_def("""
    CREATE OR REPLACE FUNCTION ai_med.przyjmowanie_leku(
    
    ) 
    
    RETURNS numeric AS $$
    
    DECLARE total integer;

    BEGIN
        select count(*) into total from ai_med.pacjent;

        RETURN total;
    
    END; $$
    LANGUAGE plpgsql;
    """,
commit = True)

In [217]:
function_tester("select ai_med.przyjmowanie_leku")

  przyjmowanie_leku
0             50091


In [218]:
function_def("""
    CREATE OR REPLACE FUNCTION ai_med.przyjmowanie_leku(
    
    ) 
    
    RETURNS numeric AS $$
    
    DECLARE total integer;

    BEGIN
        select count(*) into total from ai_med.pacjent;

        RETURN total;
    
    END; $$
    LANGUAGE plpgsql;
    """,
commit = True)

In [219]:
function_tester("select ai_med.przyjmowanie_leku")

  przyjmowanie_leku
0             50091


# TRIGGERS

In [220]:
def trigger_def(func, commit = False):
    try:
        con = psycopg2.connect(
                    host = "127.0.0.11",
                    database="ZSBD_p1",
                    user = "inzdanych123",
                    password = "inzdanych123",
                    port = "5432")

        cur = con.cursor()

        for notice in con.notices:
            print(notice)

        cur.execute(func)
        if commit:
            con.commit()

        cur.close()
        con.close()

    except (Exception, psycopg2.DatabaseError) as error:
            print(error)

    finally:
        if con is not None:
            con.close()

In [221]:
trigger_def("""
CREATE TRIGGER pacjent_insert_timestamp 
    AFTER
    INSERT 
    ON ai_med.pacjent
    FOR EACH ROW EXECUTE PROCEDURE ai_med.pacjent_insert_timestamp_func();
""",
commit=True)

trigger "pacjent_insert_timestamp" for relation "pacjent" already exists



In [222]:
function_def("""
CREATE OR REPLACE FUNCTION ai_med.pacjent_insert_timestamp_func() RETURNS TRIGGER AS $pacjent_table$
   BEGIN
      INSERT INTO ai_med.pacjent_insert_timestamp_table(id, ostatni_insert_danych) VALUES (NEW.id ,date_trunc('day', current_timestamp));
      RETURN NEW;
   END;
$pacjent_table$ LANGUAGE plpgsql;
""",
commit=True)

In [223]:
do_query("""
CREATE TABLE IF NOT EXISTS ai_med.pacjent_insert_timestamp_table
(
    id serial NOT NULL,
    ostatni_insert_danych date NOT NULL,
    ostatnie_powiadomienie_dla_MLE date
);
""", 
commit=True,
print_out=True)

NOTICE:  relation "pacjent_insert_timestamp_table" already exists, skipping

'NoneType' object is not iterable


In [224]:
do_query("""
INSERT INTO ai_med.pacjent (imie, nazwisko, data_urodzenia) VALUES ('Paul', 'California', '2022-11-23');
""", 
commit=True,
print_out=True)
do_query("""
SELECT * FROM ai_med.pacjent where imie ='Paul' and nazwisko = 'California';
""", 
commit=True,
print_out=True)
do_query("""
SELECT * FROM ai_med.pacjent_insert_timestamp_table;
""", 
commit=True,
print_out=True)


'NoneType' object is not iterable
        id  imie    nazwisko data_urodzenia                       created_at
0   110732  Paul  California     2022-11-23 2022-11-23 18:55:12.167567+01:00
1   110736  Paul  California     2022-11-23 2022-11-23 21:24:50.528890+01:00
2   110737  Paul  California     2022-11-23 2022-11-23 21:25:22.232925+01:00
3   110745  Paul  California     2022-11-23 2022-11-23 22:19:13.968228+01:00
4   110747  Paul  California     2022-11-23 2022-11-23 22:21:39.707711+01:00
..     ...   ...         ...            ...                              ...
87  110828  Paul  California     2022-11-23 2022-11-23 23:20:35.745892+01:00
88  110829  Paul  California     2002-11-23 2022-11-23 23:20:36.137202+01:00
89  110830  Paul  California     2022-11-23 2022-11-24 12:41:02.877312+01:00
90  110831  Paul  California     2002-11-23 2022-11-24 12:41:03.290916+01:00
91  110832  Paul  California     2022-11-23 2022-11-24 14:14:03.726548+01:00

[92 rows x 5 columns]


       id ostatni

In [225]:
trigger_def("""
CREATE OR REPLACE TRIGGER message_to_ML_E
    BEFORE
    INSERT 
    ON ai_med.pacjent
    FOR EACH ROW EXECUTE PROCEDURE ai_med.message_to_ML_E_func();
""",
commit=True)

In [226]:
#TO W SUMIE POWINNO BYc NA KONIEC JAKIEJŚ TRANZAKCJI
#W SENSIE ŻEBY ZROBIĆ TRANZAKCJĘ, KTÓRA ZAJMIE SIĘ DAILY INSERTEM DO TABEL
function_def("""
CREATE OR REPLACE FUNCTION ai_med.message_to_ML_E_func() RETURNS TRIGGER AS $pacjent_table$
	declare
		liczba_wprowadzonych int;
	BEGIN
		SELECT count(*) into liczba_wprowadzonych from ai_med.pacjent_insert_timestamp_table 
		where ostatni_insert_danych = date_trunc('day', current_timestamp);
		IF liczba_wprowadzonych = 5
		THEN
		UPDATE ai_med.pacjent_insert_timestamp_table
		SET ostatnie_powiadomienie_dla_mle=date_trunc('minute', current_timestamp);
		raise notice 'Dzis zostalo wprowadzonych przynajmniej % nowych pacjentow, moze chcecie zrobic douczanie modelu?', 
		
		liczba_wprowadzonych + 1;
		
		END IF;
		RETURN NEW;
   	END;
$pacjent_table$ LANGUAGE plpgsql;
""",
commit=True)



In [227]:
do_query("""
INSERT INTO ai_med.pacjent (imie, nazwisko, data_urodzenia) VALUES ('Paul', 'California', '2002-11-23');
""", 
commit=True,
print_out=True)

do_query("""
SELECT * FROM ai_med.pacjent_insert_timestamp_table;
""", 
commit=True,
print_out=True)

'NoneType' object is not iterable
       id ostatni_insert_danych ostatnie_powiadomienie_dla_mle
0  110830            2022-11-24                           None
1  110831            2022-11-24                           None
2  110832            2022-11-24                           None
3  110833            2022-11-24                           None




In [228]:
# do_query("""
# DROP TABLE IF EXISTS ai_med.pacjent_insert_timestamp_table
# """, 
# commit=True,
# print_out=True)

# do_query("""
# CREATE TABLE IF NOT EXISTS ai_med.pacjent_insert_timestamp_table
# (
#     id serial NOT NULL,
#     ostatni_insert_danych date NOT NULL,
#     ostatnie_powiadomienie_dla_MLE date
# );
# """, 
# commit=True,
# print_out=True)

# PROCEDURES

In [229]:
def procedure_def(func, commit = False):
    try:
        con = psycopg2.connect(
                    host = "127.0.0.11",
                    database="ZSBD_p1",
                    user = "inzdanych123",
                    password = "inzdanych123",
                    port = "5432")

        cur = con.cursor()

        for notice in con.notices:
            print(notice)

        cur.execute(func)
        if commit:
            con.commit()

        cur.close()
        con.close()

    except (Exception, psycopg2.DatabaseError) as error:
            print(error)

    finally:
        if con is not None:
            con.close()

# 1. Inserting data using a procedure GENEROWANIE danych za pomocą procedur?

In [230]:
procedure_def(
"""
create or replace procedure ai_med.select_all_pacjenty()
language plpgsql    
as $$
declare
    msg int;
begin

    select count(*) into msg from ai_med.pacjent;

    RAISE NOTICE 'W bazie jest juz: % pacjentow!!', msg ;

end;$$
""",
commit=True)

In [231]:
do_query("""
call ai_med.select_all_pacjenty();
""", 
commit=True,
print_out=True)

NOTICE:  W bazie jest juz: 50093 pacjentow!!

'NoneType' object is not iterable
